In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
import torch
import joblib
import os

from google.colab import drive
drive.mount('/content/drive')

!pip install scikit-tensor-py3
from sktensor import dtensor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=9574781 sha256=a1bfd3e2589a31b753210c42eac51c3f4de065cdf43cac11b458f74c351780e5
  Stored in directory: /root/.cache/pip/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
  Created wheel for scipy: filename=scipy-1.3.3-cp39-cp39-linux_x86_64.whl size=54852244

In [4]:
if torch.cuda.is_available():      
    device = torch.device('cuda')
    print('hii')
else:
    device = torch.device('cpu')

hii


In [5]:
file_path_train = '/content/drive/MyDrive/MiniProject/[1109]TrainData_adjusted.csv'
file_path_test = '/content/drive/MyDrive/MiniProject/[1109]TestData_adjusted.csv'
trainx_df = pd.read_csv(file_path_train)
testx_df = pd.read_csv(file_path_test)


In [6]:
# Encode gender feature

conditions = [
    (trainx_df['gender'] == 'male'),
    (trainx_df['gender'] == 'female'),
    (trainx_df['gender'] == 'unknown')]

choices = [0, 1, 2]

trainx_df['genderId'] = np.select(conditions, choices, default=0)

conditions = [
    (testx_df['gender'] == 'male'),
    (testx_df['gender'] == 'female'),
    (testx_df['gender'] == 'unknown')]

testx_df['genderId'] = np.select(conditions, choices, default=0)

In [7]:
# Encode sport feature

conditions = [
    (trainx_df['sport'] == 'run'),
    (trainx_df['sport'] == 'bike'),
    (trainx_df['sport'] == 'mountain bike')]

choices = [0, 1, 2]

trainx_df['sportId'] = np.select(conditions, choices, default=0)

conditions = [
    (testx_df['sport'] == 'run'),
    (testx_df['sport'] == 'bike'),
    (testx_df['sport'] == 'mountain bike')]

testx_df['sportId'] = np.select(conditions, choices, default=0)

In [8]:
# # Fit onehot encoder for categorical features

OneHot_enc = OneHotEncoder()
OneHot_enc.fit(pd.concat([trainx_df[['genderId', 'sportId']], 
                          testx_df[['genderId', 'sportId']]]))

# # Save onehot encoder

file_path = '/content/drive/MyDrive/OneHotEncoder.m'

joblib.dump(OneHot_enc, file_path)

['/content/drive/MyDrive/OneHotEncoder.m']

In [9]:
# Load onehot encoder for categorical features

dir = '/content/drive/MyDrive/OneHotEncoder.m'

OneHot_enc = joblib.load(dir)

# Load user embedding and route embedding

user_dir = '/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/entity embedding/userEmbed_tensorD_13.m'
route_dir = '/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/entity embedding/routeEmbed_tensorD_13.m'

user_embed_df = joblib.load(user_dir)
route_embed_df = joblib.load(route_dir)

In [10]:
# Define model

class DisReg_MLP_2Layer(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, p):
        super(DisReg_MLP_2Layer, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, 1)

        self.act_1 = nn.ReLU()
        self.act_2 = nn.Sigmoid()

        self.drop_1 = nn.Dropout(p)

    def forward(self, data):

        output = self.fc1(data)
        output = self.act_1(output)
        output = self.drop_1(output)

        output = self.fc2(output)
        output = self.act_2(output)

        return output


# Define function to load model

def load_model(filename, model):

    DATA_PATH = '/content/drive/MyDrive/MiniProject/Personalized_Intelligent_Fitness_Recommender-main/workout distance prediction model/scaler_model/'+filename

    checkpoint = torch.load(DATA_PATH)
    model.load_state_dict(checkpoint['best_model_state_dict'])

    return model


# Load Model

LOAD_MODEL_NAME = 'DisReg_MLP_2L_13'
INPUT_DIM = 34
HIDDEN_DIM = 64
DROP_OUT = 0.2

model = DisReg_MLP_2Layer(
    INPUT_DIM, HIDDEN_DIM, DROP_OUT).to(device)
model = load_model(LOAD_MODEL_NAME, model)

In [11]:
file_path_test = '/content/drive/MyDrive/MiniProject/[1109]TestData_adjusted.csv'
cols = ['id', 'userId', 'gender', 'sport',
        'calories', 'Route_id', 'distance_adjusted_sum']

test_df = pd.read_csv(file_path_test, usecols=cols, nrows=500)

In [12]:
display(test_df.head(10))

,id,userId,gender,sport,calories,Route_id,distance_adjusted_sum
0,337727321,13577416,male,run,459.000,10,5.42
1,111418393,3235195,male,run,868.000,10,9.99
2,153749422,3714939,male,bike,296.000,2,15.06
3,566966743,1520156,male,run,658.000,10,12.95
4,438071637,11884453,male,run,617.000,10,10.33
5,84694591,260784,male,bike,1645.500,0,65.62
6,631497044,11737814,male,bike,149.995,2,4.62
7,481169322,732008,male,run,825.000,9,12.15
8,310128604,3680369,male,run,770.000,10,10.19
9,400737405,14370069,male,run,471.000,8,7.52


In [13]:
# Take User Inputs

USER_ID = 11884453
GENDER = 'male'
SPORT = 'run'
CALORIES = 617.0	
WORKOUT_ID = 	438071637

In [14]:
# find Route_id based on workout id
Route_id = test_df.loc[test_df.id == WORKOUT_ID, 'Route_id'].to_numpy()[0]
# find route total distance based on workout id
distance_adjusted_sum = test_df.loc[test_df.id ==
                                    WORKOUT_ID, 'distance_adjusted_sum'].to_numpy()[0]
# find userEmbedding based on userId
user_embed = np.array(
    user_embed_df[user_embed_df.userId == USER_ID].userEmbed.values[0])
# find routeEmbedding based on routeId
route_embed = np.array(
    route_embed_df[route_embed_df.Route_id == Route_id].routeEmbed.values[0])

In [15]:
test_df1=test_df.copy()

In [16]:
display(test_df1.head(5))

,id,userId,gender,sport,calories,Route_id,distance_adjusted_sum
0,337727321,13577416,male,run,459.0,10,5.42
1,111418393,3235195,male,run,868.0,10,9.99
2,153749422,3714939,male,bike,296.0,2,15.06
3,566966743,1520156,male,run,658.0,10,12.95
4,438071637,11884453,male,run,617.0,10,10.33


In [17]:
scalers = {}

# specify the features to scale
features = ['calories', 'distance_adjusted_sum']

# fit a scaler object to each feature and store it in the scalers dictionary
for feature in features:
    # get the maximum and minimum values of the feature in the dataframe
    max_val = test_df1[feature].max()
    min_val = test_df1[feature].min()
    
    # create a scaler object and fit it to the feature values
    scaler = MinMaxScaler()
    scaler.fit(test_df1[feature].values.reshape(-1, 1))
    
    # store the scaler object in the scalers dictionary
    scalers[feature] = scaler

In [18]:
import warnings

In [19]:
def data_process(gender, sport, calories, Route_id, distance_adjusted_sum, user_embed, route_embed):
    # label encode gender

    choices = [0, 1, 2]
    conditions = [
        (gender == 'male'),
        (gender == 'female'),
        (gender == 'unknown')]

    genderId = np.select(conditions, choices, default=0)

    # label encode sport

    conditions = [
        (sport == 'run'),
        (sport == 'bike'),
        (sport == 'mountain bike')]

    sportId = np.select(conditions, choices, default=0)

    # scale calories
    scaler_calorie = scalers['calories']
    scaled_calories_data = scaler_calorie.transform([[calories]])
    calories_scaled = scaled_calories_data[0][0]

    # scale distance
    scaler_distance = scalers['distance_adjusted_sum']
    scaled_distance_data = scaler_distance.transform([[distance_adjusted_sum]])
    total_distance_scaled = scaled_distance_data[0][0]

    # one hot encode genderId and sportId
    gender_sport_onehot = OneHot_enc.transform(
        np.hstack((genderId, sportId)).reshape(1, -1)).toarray()[0]
    # concatenate input features into numpy array
    data_input = np.hstack((calories_scaled, total_distance_scaled,
                            user_embed, route_embed, gender_sport_onehot))

    return data_input


data_input = data_process(
    GENDER,
    SPORT,
    CALORIES,
    Route_id,
    distance_adjusted_sum,
    user_embed,
    route_embed)

In [20]:
model.eval()
with torch.no_grad():
    pred = model(torch.Tensor(data_input).to(device)).item()

scaler_distance = scalers['distance_adjusted_sum']
# reshape the predicted distance to a 2D array with one row and one column
pred_2d = np.array([[pred]])
# convert predicted distance to the min-max scaled range
pred_scaled = scaler_distance.inverse_transform(pred_2d)
# extract the unscaled distance value from the scaled range
pred_unscaled = pred_scaled[0][0]
print('The predicted distance is:{:.4} km'.format(pred_unscaled))
print('Workout route total distance: {:.4} km'.format(distance_adjusted_sum))

The predicted distance is:9.818 km
Workout route total distance: 10.33 km


In [ ]:
display(test_df1.head(1))

,id,userId,gender,sport,calories,Route_id,distance_adjusted_sum
0,337727321,13577416,male,run,459.0,10,5.42


In [22]:
# import the required library
from sklearn.metrics import mean_squared_error

# create a list to store the predicted distances and actual distances
pred_distances = []
actual_distances = []

# loop over each row in the test dataset
for index, row in test_df.iterrows():

    # extract the input variables from the row
    user_id = row['userId']
    gender = row['gender']
    sport = row['sport']
    calories = row['calories']
    workout_id = row['id']
    route_id = row['Route_id']
    distance_adjusted_sum = row['distance_adjusted_sum']
    user_embed = np.array(user_embed_df[user_embed_df.userId == user_id].userEmbed.values[0])
    route_embed = np.array(route_embed_df[route_embed_df.Route_id == route_id].routeEmbed.values[0])

    # preprocess the input variables
    data_input = data_process(gender, sport, calories, route_id, distance_adjusted_sum, user_embed, route_embed)

    # make the distance prediction
    model.eval()
    with torch.no_grad():
        pred_distance = model(torch.Tensor(data_input).to(device)).item()

    # unscale the predicted distance
    scaler_distance = scalers['distance_adjusted_sum']
    pred_distance_2d = np.array([[pred_distance]])
    pred_distance_scaled = scaler_distance.inverse_transform(pred_distance_2d)
    pred_distance_unscaled = pred_distance_scaled[0][0]

    # store the predicted distance and actual distance in the respective lists
    pred_distances.append(pred_distance_unscaled)
    actual_distances.append(row['distance_adjusted_sum'])

# calculate the root mean squared error (RMSE)
rmse = mean_squared_error(actual_distances, pred_distances, squared=False)

print('Root Mean Squared Error (RMSE): {:.2f}'.format(rmse))


Root Mean Squared Error (RMSE): 1.75


In [23]:
# create a list to store the predicted distances and actual distances
pred_distances = []
actual_distances = []

# loop over each row in the test dataset
for index, row in test_df.iterrows():

    # extract the input variables from the row
    user_id = row['userId']
    gender = row['gender']
    sport = row['sport']
    calories = row['calories']
    workout_id = row['id']
    route_id = row['Route_id']
    distance_adjusted_sum = row['distance_adjusted_sum']
    user_embed = np.array(user_embed_df[user_embed_df.userId == user_id].userEmbed.values[0])
    route_embed = np.array(route_embed_df[route_embed_df.Route_id == route_id].routeEmbed.values[0])

    # preprocess the input variables
    data_input = data_process(gender, sport, calories, route_id, distance_adjusted_sum, user_embed, route_embed)

    # make the distance prediction
    model.eval()
    with torch.no_grad():
        pred_distance = model(torch.Tensor(data_input).to(device)).item()

    # unscale the predicted distance
    scaler_distance = scalers['distance_adjusted_sum']
    pred_distance_2d = np.array([[pred_distance]])
    pred_distance_scaled = scaler_distance.inverse_transform(pred_distance_2d)
    pred_distance_unscaled = pred_distance_scaled[0][0]

    # store the predicted distance and actual distance in the respective lists
    pred_distances.append(pred_distance_unscaled)
    actual_distances.append(row['distance_adjusted_sum'])

# calculate the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((np.array(actual_distances) - np.array(pred_distances)) / np.array(actual_distances))) * 100

accuracy = 100 - mape
print('Mean absolute percentage error (MAPE): {:.2f}%'.format(mape))
print('Accuracy: {:.2f}%'.format(accuracy))

Mean absolute percentage error (MAPE): 4.98%
Accuracy: 95.02%
